# Post-Process FATES or CLM Ensemble

In [6]:
import os
import numpy as np
import xarray as xr
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

import fates_calibration_library.analysis_functions as analysis
import fates_calibration_library.utils as utils

In [2]:
# Setup PBSCluster
cluster = PBSCluster(
    cores=1,                                                   # The number of cores you want
    memory='25GB',                                             # Amount of memory
    processes=1,                                               # How many processes
    queue='casper',                                            # The type of queue to utilize
    local_directory='/glade/work/afoster',                     # Use your local directory
    resource_spec='select=1:ncpus=1:mem=25GB',                 # Specify resources
    log_directory='/glade/derecho/scratch/afoster/dask_logs',  # log directory
    account='P08010000',                                       # Input your project ID here
    walltime='02:00:00',                                       # Amount of wall time
    interface='ext')                                           # Interface to use

/glade/work/afoster/conda-envs/fates_calibration/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37493 instead
  warnings.warn(


In [3]:
cluster.scale(30)
dask.config.set({
    'distributed.dashboard.link': 'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
})
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/37493/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/37493/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.173:41179,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/37493/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Set Up

First specify history variables and variables to process

In [ ]:
fates_vars = ['FATES_FRACTION', 'FATES_GPP', 'FATES_LAI', 'QVEGE',
              'QVEGT', 'EFLX_LH_TOT', 'FSH', 'FSR', 'FSDS', 'FSA',
              'FIRE', 'FLDS', 'FCTR', 'FCEV', 'FGEV', 'BTRANMN',
              'FGR', 'SOILWATER_10CM', 'TWS', 'QRUNOFF', 'SNOWDP',
              'TV', 'TG', 'TSA', 'TBOT']

clm_vars = ['FPSN', 'TLAI', 'QVEGE', 'QVEGT', 'EFLX_LH_TOT', 'FSH',
            'FSR', 'FSDS', 'FSA', 'FIRE', 'FLDS', 'FCTR', 'FCEV',
            'FGEV', 'BTRANMN', 'FGR', 'SOILWATER_10CM', 'TWS',
            'QRUNOFF', 'SNOWDP', 'TV', 'TG', 'TSA', 'TBOT']

out_vars = ['GPP', 'LAI', 'EFLX_LH_TOT', 'FSH', 'EF', 'SOILWATER_10CM', 'ASA',
            'FSR', 'FSA', 'FIRE', 'RLNS', 'RN', 'BTRANMN', 'TV']

### Files Needed

In [8]:
# sparse grid whittaker biomes
whittaker_ds = xr.open_dataset('/glade/work/afoster/FATES_calibration/observations/whittaker/whitkey.nc')

# sparsegrid landarea - needed for unit conversion
land_area_file = '/glade/work/afoster/FATES_calibration/CLM5PPE/postp/sparsegrid_landarea.nc'
land_area = xr.open_dataset(land_area_file).landarea

# config file with conversion information in it
var_config = '/glade/work/afoster/FATES_calibration/scripts/configs/model_conversion.yaml'
var_dict = utils.get_config_file(var_config)

In [ ]:
def write_history_files(top_dir, data_vars, param_key, postp_dir, fates=False, fates_params=False):
    
    dirs = sorted(os.listdir(top_dir))
    keys_finished = []

    for dir in dirs:
        ensemble = dir.split('_')[-1]
        out_file = os.path.join(postp_dir, f"{dir}.nc")
        
        if not os.path.isfile(out_file):
            
            ds = post_process(os.path.join(top_dir, dir), data_vars, param_key, fates=fates, fates_params=fates_params)
            if ds is not None:
                ds.to_netcdf(out_file)
                keys_finished.append(ensemble)
        else:
            keys_finished.append(ensemble)
    
    all_keys = param_key.key.values
    not_finished = []
    for key in all_keys:
        if key not in keys_finished:
            not_finished.append(key)
    return not_finished

In [ ]:
#sparse_fates_dir = '/glade/derecho/scratch/afoster/archive/ctsm60SP_fates_sparse_vai/lnd/hist'
#sparse_fates_ds = analysis.post_process_ds(sparse_fates_dir, fates_vars, whittaker_ds,
#                                     years=[2000, 2015], run_dict={'filter_nyears': 20})

In [10]:
top_dir = '/glade/derecho/scratch/afoster/FATES_calibration/fates_oaat/archive'
dirs = sorted(os.listdir(top_dir))

In [12]:
postp_dir = '/glade/work/afoster/FATES_calibration/history_files/fates_oaat'

In [15]:
for dir in dirs:
    ensemble = dir.split('_')[-1]
    out_file = os.path.join(postp_dir, f"{dir}.nc")
    if os.path.isfile(out_file) and not clobber:
        
    if not os.path.isfile(out_file):

/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_001.nc
/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_002.nc
/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_003.nc
/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_004.nc
/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_005.nc
/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_006.nc
/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_007.nc
/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_008.nc
/glade/work/afoster/FATES_calibration/history_files/fates_oaat/ctsm60SP_fates_sparse_vai_FATES_OAAT_009.nc
/glade/work/afoster/FATES_calibration

In [ ]:
out